In [27]:
import numpy as np
from treelib import Tree
import copy
import random
import spams

class flowtree():
    def __init__(self, samples_A, samples_B):
        """
         Parameter
         ----------
         samples_A :
             first set of supports
         samples_B :
             second set of supports
         M : 
             cost matrix between samples_A and samples_B
         """
        # step1: build quadtree
        self.tree = self.build_quadtree(np.vstack([samples_A, samples_B]), random_shift=True, width=None, origin=None)
        self.flow_matrix = np.zeros((len(samples_A), len(samples_B)))
        # self.tree.show()
    
    def result(self, M):
        # step2: compute flow
        self.compute_flow2(self.tree.get_node(self.tree.root), self.flow_matrix)
        cost = np.sum(M * self.flow_matrix)
        return cost, self.flow_matrix


    def _build_quadtree(self, X, origin, remaining_idx, width):
        d = X.shape[1]  # dimension
        m = len(remaining_idx)  # number of samples (i.e., support size)

        tree = Tree()
        tree.create_node(data=None)

        loc = np.zeros(m).tolist()

        # divide the hypercube, and obtain which hypercube a point belong to.
        for i in range(len(remaining_idx)):
            for j in range(d):
                if X[remaining_idx[i]][j] > origin[j]:
                    loc[i] += 2 ** j

        child = list(set(loc))
        child_set = [[] for _ in range(len(child))]
        origin_set = []

        for i in range(len(child)):
            new_origin = np.zeros_like(origin)
            for j in range(d):
                if int(child[i]) & (2 ** j) != 0:
                    new_origin[j] = copy.deepcopy(origin[j]) + width / 2.0
                else:
                    new_origin[j] = copy.deepcopy(origin[j]) - width / 2.0
            origin_set.append(new_origin)

        for i in range(m):
            child_set[child.index(loc[i])].append(remaining_idx[i])

        for i in range(len(child)):
            if len(child_set[i]) == 1:
                tree.create_node(f"{X[child_set[i]][0]}", parent=tree.root, data=child_set[i][0])
            else:
                subtree = self._build_quadtree(X, origin_set[i], child_set[i], width / 2.0)
                tree.paste(tree.root, subtree)

        return tree

    def build_quadtree(self, X, random_shift=True, width=None, origin=None):
        """
        Assume that X[i] in [0, width]^d.
        """
        #np.random.seed(0)
        if random_shift:
            # check the assumption.
            if np.min(X) < 0:
                print("Warn : Assumption, lower than zero")
                X = X - np.min(X)
            elif np.min(X) != 0:
                X = X - np.min(X)
                print("Warn : Assumption, larger than zero")

            width = np.max(X)
            origin = np.random.uniform(low=0.0, high=width, size=X.shape[1])

        remaining_idx = [i for i in range(X.shape[0])]

        return self._build_quadtree(X, origin, remaining_idx, width)
    
    def compute_flow(self, node, flow_matrix):
        samples_A_count = flow_matrix.shape[0]
        samples_B_count = flow_matrix.shape[1]
        mu = 1.0 / samples_A_count
        nu = 1.0 / samples_B_count

        unmatched_mu = np.zeros(samples_A_count)
        unmatched_nu = np.zeros(samples_B_count)

        if node.is_leaf():
            data = node.data
            if data < samples_A_count:
                unmatched_mu[data] = mu
            else:
                unmatched_nu[data - samples_A_count] = nu
            return unmatched_mu, unmatched_nu

        for child in node.successors(self.tree.identifier):
            child_node = self.tree.get_node(child)
            child_unmatched_mu, child_unmatched_nu = self.compute_flow(child_node, flow_matrix)

            unmatched_mu += child_unmatched_mu
            unmatched_nu += child_unmatched_nu

        i_indices, = np.nonzero(unmatched_mu)
        j_indices, = np.nonzero(unmatched_nu)

        for i in i_indices:
            for j in j_indices:
                flow_value = min(unmatched_mu[i], unmatched_nu[j])
                flow_matrix[i, j] += flow_value

                unmatched_mu[i] -= flow_value
                unmatched_nu[j] -= flow_value

                if unmatched_mu[i] == 0:
                    break

        return unmatched_mu, unmatched_nu
    
    def compute_flow2(self, node, flow_matrix):
        samples_A_count, samples_B_count = flow_matrix.shape
        mu = 1.0 / samples_A_count
        nu = 1.0 / samples_B_count

        unmatched_mu = np.zeros(samples_A_count)
        unmatched_nu = np.zeros(samples_B_count)

        if node.is_leaf():
            data = node.data
            if data < samples_A_count:
                unmatched_mu[data] = mu
            else:
                unmatched_nu[data - samples_A_count] = nu
            return unmatched_mu, unmatched_nu

        for child in node.successors(self.tree.identifier):
            child_node = self.tree.get_node(child)
            child_unmatched_mu, child_unmatched_nu = self.compute_flow2(child_node, flow_matrix)
            unmatched_mu += child_unmatched_mu
            unmatched_nu += child_unmatched_nu

        i_indices, = np.nonzero(unmatched_mu)
        j_indices, = np.nonzero(unmatched_nu)

        for i in i_indices:
            for j in j_indices:
                flow_value = min(unmatched_mu[i], unmatched_nu[j])
                if flow_value == 0:
                    continue

                flow_matrix[i, j] += flow_value
                unmatched_mu[i] -= flow_value
                unmatched_nu[j] -= flow_value

                if unmatched_mu[i] == 0:
                    break

        return unmatched_mu, unmatched_nu


In [36]:
import numpy as np
import time
from scipy.spatial.distance import cdist
import ot
import ot_estimators


def generate_test_data(samples_A_count, samples_B_count, features):
    samples_A = np.random.rand(samples_A_count, features)
    samples_B = np.random.rand(samples_B_count, features)
    return samples_A, samples_B

if __name__ == "__main__":
    samples_A_count = 1000
    samples_B_count = 2000
    features = 100
    
    samples_A, samples_B= generate_test_data(samples_A_count, samples_B_count, features)
    dataset = [
        [(i, 1/samples_A_count) for i in range(samples_A_count)],  # 第一个数据点
        [(i+samples_A_count, 1/samples_B_count) for i in range(samples_B_count)]   # 第二个数据点
    ]
    start_time0 = time.time()
    M = cdist(samples_A, samples_B, 'euclidean')
    end_time0 = time.time()
    start_time00 = time.time()
    ote = ot_estimators.OTEstimators()
    vocab = np.vstack([samples_A, samples_B])
    vocab = vocab.astype(np.float32)
    ote.load_vocabulary(vocab)
    ote.load_dataset(dataset)
    emd, flow_matrix = ote.compute_flowtree_emd_between_dataset_points()
    end_time00 = time.time()
    start_time = time.time()
    tree = flowtree(samples_A, samples_B)
    end_time = time.time()
    start_time1 = time.time()
    cost, flow = tree.result(M)
    end_time1 = time.time()
    start_time2 = time.time()
    cost2 = ot.emd2([], [], M)
    end_time2 = time.time()
    start_time3 = time.time()
    cost3 = ot.sinkhorn2([], [], M, reg=100)
    end_time3 = time.time()
    print('flowtree_C:', emd, ', flowtree_python:', cost, ', emd:', cost2, ', sinkhorn:', cost3)
    print(f"compute cost matrix took {end_time0 - start_time0:.6f} seconds")
    print(f"flowtree_C took {end_time00 - start_time00:.6f} seconds")
    print(f"flowtree_python took {end_time1 - start_time:.6f} seconds, where construct tree took {end_time - start_time:.6f} seconds")
    # print(f"construct flowtree took {end_time - start_time:.6f} seconds")
    # print(f"compute flow in flowtree took {end_time1 - start_time1:.6f} seconds")
    print(f"EMD took {end_time2 - start_time2:.6f} seconds")
    print(f"sinkhorn took {end_time3 - start_time3:.6f} seconds")


Dataset loaded. Size: 2
Warn : Assumption, larger than zero
flowtree_C: 4.086383819580078 , flowtree_python: 4.075169503978931 , emd: 3.3861483388050817 , sinkhorn: 4.078291436842288
compute cost matrix took 0.124177 seconds
flowtree_C took 0.165965 seconds
flowtree_python took 3.258551 seconds, where construct tree took 2.711091 seconds
EMD took 0.315232 seconds
sinkhorn took 0.067134 seconds


In [23]:
import numpy as np
import ot_estimators  # 导入你的 C++ 模块

# 创建 OTEstimators 对象
ote = ot_estimators.OTEstimators()

def generate_test_data(samples_A_count, samples_B_count, features):
    samples_A = np.random.rand(samples_A_count, features)
    samples_B = np.random.rand(samples_B_count, features)
    return samples_A, samples_B

# 加载词汇表（vocab）
samples_A_count = 1000
samples_B_count = 2000
features = 100

samples_A, samples_B= generate_test_data(samples_A_count, samples_B_count, features)

vocab = np.vstack([samples_A, samples_B])
# vocab = np.array([
#     [1, 2],
#     [3, 4],
#     [5, 6],
#     [13, 14],
# ])
vocab = vocab.astype(np.float32)
# ote.load_vocabulary(vocab)

# dataset = [[(0,0.5), (1, 0.5)], [(2, 0.5), (3, 0.5)]]

# 加载数据集（dataset），假设数据集只包含两个数据点
dataset = [
    [(i, 1/samples_A_count) for i in range(samples_A_count)],  # 第一个数据点
    [(i+samples_A_count, 1/samples_B_count) for i in range(samples_B_count)]   # 第二个数据点
]
# dataset = dataset.astype(np.float32)
# print(dataset)
ote.load_vocabulary(vocab)
ote.load_dataset(dataset)


Dataset loaded. Size: 2


In [24]:
from scipy.spatial.distance import cdist
import ot
# M = cdist([[1, 2],[3, 4],], [[5, 6],[13, 14],], 'euclidean')
M = cdist(samples_A, samples_B, 'euclidean')
# print(ot.emd([], [], M))
print(ot.emd2([], [], M))

3.387729025133421


In [25]:
# 计算数据集中两个数据点之间的 FlowTree EMD 和流矩阵
emd, flow_matrix = ote.compute_flowtree_emd_between_dataset_points()
# print(np.sum(flow_matrix*M))

# 输出结果
print("EMD between two dataset points:", emd)
# print("Flow matrix:", flow_matrix)

EMD between two dataset points: 4.077254772186279
